In [1]:
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkConf

from pathlib import Path

In [ ]:
warehouse = str(Path.home() / "temp" / "iceberg" / "data" / "spark-warehouse")

In [3]:
conf = (
    SparkConf()
    .setAppName("TEEHR")
    .setMaster("local[*]")
    .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")
    .set("spark.sql.execution.arrow.pyspark.enabled", "true")
    .set("spark.sql.session.timeZone", "UTC")

    .set("spark.sql.package", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.0")
    .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    # .set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
    # .set("spark.sql.catalog.spark_catalog.type", "hive")

    .set("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.sql.catalog.local.type", "hadoop")
    .set("spark.sql.catalog.local.warehouse", warehouse)

    .set("spark.driver.host", "localhost")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 08:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark.sql("
spark.sql("CREATE TABLE local.table (id bigint, data string) USING iceberg;")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `table` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [ ]:
spark.sql("INSERT INTO local.db.table VALUES (1, 'a'), (2, 'b'), (3, 'c')")

In [5]:
spark.sql("SELECT * FROM local.db.table").show()

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
+---+----+



In [11]:
df = spark.table("local.table")
df.count()

0

In [7]:
spark.sql("SELECT * FROM local.db.table.snapshots;").show()

+--------------------+-------------------+---------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+---------+---------+--------------------+--------------------+
|2025-05-20 00:48:...|1791743308109392874|     NULL|   append|/Users/mdenno/tem...|{spark.app.id -> ...|
+--------------------+-------------------+---------+---------+--------------------+--------------------+



In [ ]:
spark.sql("DROP TABLE local.db.primary_timeseries;")
spark.sql("""
    CREATE TABLE local.db.primary_timeseries (
        reference_time timestamp,
        value_time timestamp,
        configuration_name string,
        unit_name string,
        variable_name string,
        value float,
        location_id string
    ) USING iceberg PARTITIONED BY (configuration_name, variable_name, reference_time);
""")

In [ ]:
spark.read.format("parquet").load("/Users/mdenno/repos/teehr/tests/data/two_locations/two_locations.parquet").show()

In [9]:
# spark.sql("DROP TABLE local.db.location;")
spark.sql("""
    CREATE TABLE local.db.location (
        id string,
        name string,
        geom binary
    ) USING iceberg;
""")

DataFrame[]

In [12]:
spark.stop()